In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import json
from bs4 import BeautifulSoup
from splinter import Browser
import csv
import time
import random
import pandas as pd
from datetime import datetime
import re
from dateutil import tz
from ast import literal_eval
from selenium import webdriver
import pickle
from os import listdir
from os.path import isfile, join
import os

In [24]:
import urllib.request, json 
myURL = "https://raw.githubusercontent.com/chinese-poetry/chinese-poetry/master/json/poet.tang.7000.json"

with urllib.request.urlopen(myURL) as url:
    data = json.loads(url.read().decode())
#     print(data)

[{'paragraphs': ['扶蘇秦帝子，舉代稱其賢。', '百萬猶在握，可爭天下權。', '束身就一劒，壯志皆棄捐。', '塞下有遺跡，千齡人共傳。', '疎蕪盡荒草，寂歷空寒煙。', '到此盡垂淚，非我獨潸然。'], 'author': '陶翰', 'title': '經殺子谷', 'strains': ['平平平仄仄，仄仄○○平。', '仄仄○仄仄，仄平平仄平。', '仄平仄仄仄，仄仄平仄平。', '仄仄仄○仄，平平平仄○。', '○平仄平仄，仄仄○平平。', '仄仄仄平仄，平仄仄○平。']}, {'paragraphs': ['夜來三渚風，晨過臨淮島。', '湖中海氣白，城上楚雲早。', '鱗鱗魚浦帆，漭漭蘆洲草。', '川路日浩蕩，惄焉心如擣。', '且言任倚伏，何暇念枯槁。', '范子名屢移，蘧公志常保。', '古人去已久，此理今難道。'], 'author': '陶翰', 'title': '早過臨淮', 'strains': ['仄平○仄平，平○○平仄。', '平○仄仄仄，平仄仄平仄。', '平平平仄○，仄仄平平仄。', '平仄仄仄仄，仄平平○仄。', '○平平仄仄，平仄仄平仄。', '仄仄平仄平，平平仄平仄。', '仄平仄仄仄，仄仄平○仄。']}, {'paragraphs': ['艤棹乘早潮，潮來如風雨。', '樟臺忽已隱，界峯莫及覩。', '崩騰心爲失，浩蕩目無主。', '豗{忄畫}浪始聞，漾漾入魚浦。', '雲景共澄霽，江山相吞吐。', '偉哉造化工，此事從終古。', '流沫誠足誡，商歌調易苦。', '頗因忠信全，客心猶栩栩。'], 'author': '陶翰', 'title': '乘潮至漁浦作', 'strains': ['仄仄○仄平，平平○平仄。', '平平仄仄仄，仄平仄仄仄。', '平平平平仄，仄仄仄平仄。', '平仄○仄○，仄仄仄平仄。', '平仄仄平仄，平平○平仄。', '仄平仄仄平，仄仄○平仄。', '平仄平仄仄，平平○仄仄。', '○平平仄平，仄平○仄仄。']}, {'paragraphs': ['山月松篠下，月明山景鮮。', '聊爲高秋酌，復此清夜弦。', '晤語方獲志，栖心亦彌年。', '尚言興未逸，更理逍遙篇。'], 'author': '陶翰', 'title': '秋山夕興', 'strains': ['平仄平

In [35]:
data[10]['strains']

['平平仄平仄，仄仄仄平平。', '平仄平平仄，平平仄仄平。']

In [41]:
len(data)

1000

In [42]:
len(data[0]['paragraphs'])

6

In [43]:
data[0]['paragraphs']

['扶蘇秦帝子，舉代稱其賢。',
 '百萬猶在握，可爭天下權。',
 '束身就一劒，壯志皆棄捐。',
 '塞下有遺跡，千齡人共傳。',
 '疎蕪盡荒草，寂歷空寒煙。',
 '到此盡垂淚，非我獨潸然。']

In [48]:
data[0]

{'author': '陶翰',
 'paragraphs': ['扶蘇秦帝子，舉代稱其賢。',
  '百萬猶在握，可爭天下權。',
  '束身就一劒，壯志皆棄捐。',
  '塞下有遺跡，千齡人共傳。',
  '疎蕪盡荒草，寂歷空寒煙。',
  '到此盡垂淚，非我獨潸然。'],
 'strains': ['平平平仄仄，仄仄○○平。',
  '仄仄○仄仄，仄平平仄平。',
  '仄平仄仄仄，仄仄平仄平。',
  '仄仄仄○仄，平平平仄○。',
  '○平仄平仄，仄仄○平平。',
  '仄仄仄平仄，平仄仄○平。'],
 'title': '經殺子谷'}

In [49]:
import re

def cleanLine(line):    
    line = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）：；《）《》“”()»〔〕-]+", "",line)
    return(line)

In [60]:
import numpy as np

author_list = []
title_list = []
cha_list = []
# strain_list = []
# author_id = []
# title_id = []
char_id = []

for poem in data:
#     author_list.append(poem['author'])
#     title_list.append(poem['title'])    
    i = 0
    for para in poem['paragraphs']:
        para = cleanLine(para)
        for char in para:
            author_list.append(poem['author'])
            title_list.append(poem['title'])
            cha_list.append(char)            
            char_id.append(i)
            i = i + 1
    

char_df = pd.DataFrame(np.column_stack([author_list, title_list, cha_list, char_id]), 
                               columns=['author', 'title', 'char', 'char_id'])
char_df.head(500)

,author,title,char,char_id
0,陶翰,經殺子谷,扶,0
1,陶翰,經殺子谷,蘇,1
2,陶翰,經殺子谷,秦,2
3,陶翰,經殺子谷,帝,3
4,陶翰,經殺子谷,子,4
5,陶翰,經殺子谷,舉,5
6,陶翰,經殺子谷,代,6
7,陶翰,經殺子谷,稱,7
8,陶翰,經殺子谷,其,8
9,陶翰,經殺子谷,賢,9


Investigate unicode encoding

In [64]:
u'\u76ca'

'益'

In [72]:
'陶翰'.encode('utf-8')

b'\xe9\x99\xb6\xe7\xbf\xb0'

In [76]:
'陶'.encode('utf-8')

b'\xe9\x99\xb6'

In [74]:
'經'.encode('utf-8')

b'\xe7\xb6\x93'

In [91]:
int.from_bytes('經殺子谷'.encode('utf-8'), byteorder='big')

71711760211115320418155147447

In [95]:
def encode(chinese_str):
    code = int.from_bytes(chinese_str.encode('utf-8'), byteorder='big')
    return(hex(code))

In [97]:
char_df['author_code'] = char_df['author'].apply(encode)
char_df['title_code'] = char_df['title'].apply(encode)
char_df

,author,title,char,char_id,author_code,title_code
0,陶翰,經殺子谷,扶,0,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
1,陶翰,經殺子谷,蘇,1,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
2,陶翰,經殺子谷,秦,2,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
3,陶翰,經殺子谷,帝,3,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
4,陶翰,經殺子谷,子,4,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
5,陶翰,經殺子谷,舉,5,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
6,陶翰,經殺子谷,代,6,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
7,陶翰,經殺子谷,稱,7,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
8,陶翰,經殺子谷,其,8,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
9,陶翰,經殺子谷,賢,9,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7


In [104]:
def rhash(n):
    return "%08x" % (n * 387420489 % 4000000000000000000000000000)

def un_rhash(h):
    return int(h, 16) * 3513180409 % 4000000000000000000000000000

hex(un_rhash(rhash(0xe999b6e7bfb0)))


'0xb5048780dfe913038bfb0'

## Need to fit the charactor gene into 64 hex number

### Two possible approaches. 1) encode the chinese characters and use these as ID 2) create an ordered ID starting from 0

In [105]:
char_df

,author,title,char,char_id,author_code,title_code
0,陶翰,經殺子谷,扶,0,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
1,陶翰,經殺子谷,蘇,1,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
2,陶翰,經殺子谷,秦,2,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
3,陶翰,經殺子谷,帝,3,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
4,陶翰,經殺子谷,子,4,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
5,陶翰,經殺子谷,舉,5,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
6,陶翰,經殺子谷,代,6,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
7,陶翰,經殺子谷,稱,7,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
8,陶翰,經殺子谷,其,8,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7
9,陶翰,經殺子谷,賢,9,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7


In [109]:
char_df = char_df.assign(author_id2=(char_df['author']).astype('category').cat.codes)
char_df = char_df.assign(title_id2=(char_df['title']).astype('category').cat.codes)
char_df

,author,title,char,char_id,author_code,title_code,author_id2,title_id2
0,陶翰,經殺子谷,扶,0,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
1,陶翰,經殺子谷,蘇,1,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
2,陶翰,經殺子谷,秦,2,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
3,陶翰,經殺子谷,帝,3,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
4,陶翰,經殺子谷,子,4,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
5,陶翰,經殺子谷,舉,5,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
6,陶翰,經殺子谷,代,6,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
7,陶翰,經殺子谷,稱,7,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
8,陶翰,經殺子谷,其,8,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
9,陶翰,經殺子谷,賢,9,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561


In [130]:
author_df = char_df[['author', 'title']].drop_duplicates()['author'].value_counts().reset_index()
author_df.columns = ['author', 'poem_count']
author_df

,author,poem_count
0,劉長卿,510
1,孟浩然,268
2,李白,68
3,蕭穎士,41
4,李華,29
5,孟雲卿,18
6,崔曙,15
7,王翰,14
8,顏真卿,10
9,賀蘭進明,7


In [150]:
author_df = char_df[['author', 'title']].drop_duplicates().groupby(['author']).size().reset_index()
author_df.columns = ['author', 'author_poem_count']
author_df

,author,author_poem_count
0,劉長卿,510
1,孟浩然,268
2,孟雲卿,18
3,崔曙,15
4,庾光先,1
5,張巡,2
6,張抃,1
7,李希仲,3
8,李白,68
9,李華,29


In [151]:
poem_df = char_df.groupby(['author', 'title']).size().reset_index()
poem_df.columns = ['author', 'title', 'poem_char_count']
poem_df

,author,title,poem_char_count
0,劉長卿,七里灘重送,28
1,劉長卿,三月李明府後亭泛舟,56
2,劉長卿,上巳日越中與鮑侍郎泛舟耶溪,56
3,劉長卿,上湖田館南樓憶朱晏,80
4,劉長卿,上陽宮望幸,56
5,劉長卿,九日岳陽待黃遂張渙,80
6,劉長卿,九日登李明府北樓,40
7,劉長卿,九日題蔡國公主樓,60
8,劉長卿,京口懷洛陽舊居兼寄廣陵二三知己,100
9,劉長卿,代邊將有懷,40


In [152]:
char_df[char_df['title']=='入白沙渚夤緣二十五里至石窟山下懷天台陸山人']

,author,title,char,char_id,author_code,title_code,author_id2,title_id2
15045,劉長卿,入白沙渚夤緣二十五里至石窟山下懷天台陸山人,遠,0,0xe58a89e995b7e58dbf,0xe585a5e799bde6b299e6b89ae5a4a4e7b7a3e4ba8ce5...,0,47
15046,劉長卿,入白沙渚夤緣二十五里至石窟山下懷天台陸山人,嶼,1,0xe58a89e995b7e58dbf,0xe585a5e799bde6b299e6b89ae5a4a4e7b7a3e4ba8ce5...,0,47
15047,劉長卿,入白沙渚夤緣二十五里至石窟山下懷天台陸山人,靄,2,0xe58a89e995b7e58dbf,0xe585a5e799bde6b299e6b89ae5a4a4e7b7a3e4ba8ce5...,0,47
15048,劉長卿,入白沙渚夤緣二十五里至石窟山下懷天台陸山人,將,3,0xe58a89e995b7e58dbf,0xe585a5e799bde6b299e6b89ae5a4a4e7b7a3e4ba8ce5...,0,47
15049,劉長卿,入白沙渚夤緣二十五里至石窟山下懷天台陸山人,夕,4,0xe58a89e995b7e58dbf,0xe585a5e799bde6b299e6b89ae5a4a4e7b7a3e4ba8ce5...,0,47
15050,劉長卿,入白沙渚夤緣二十五里至石窟山下懷天台陸山人,玩,5,0xe58a89e995b7e58dbf,0xe585a5e799bde6b299e6b89ae5a4a4e7b7a3e4ba8ce5...,0,47
15051,劉長卿,入白沙渚夤緣二十五里至石窟山下懷天台陸山人,幽,6,0xe58a89e995b7e58dbf,0xe585a5e799bde6b299e6b89ae5a4a4e7b7a3e4ba8ce5...,0,47
15052,劉長卿,入白沙渚夤緣二十五里至石窟山下懷天台陸山人,行,7,0xe58a89e995b7e58dbf,0xe585a5e799bde6b299e6b89ae5a4a4e7b7a3e4ba8ce5...,0,47
15053,劉長卿,入白沙渚夤緣二十五里至石窟山下懷天台陸山人,自,8,0xe58a89e995b7e58dbf,0xe585a5e799bde6b299e6b89ae5a4a4e7b7a3e4ba8ce5...,0,47
15054,劉長卿,入白沙渚夤緣二十五里至石窟山下懷天台陸山人,遲,9,0xe58a89e995b7e58dbf,0xe585a5e799bde6b299e6b89ae5a4a4e7b7a3e4ba8ce5...,0,47


In [190]:
char_df2 = (char_df.merge(author_df, on='author', how='inner')
                 .merge(poem_df, on=['author', 'title'], how='inner')
                 .drop(['author_code', 'title_code'], axis=1))
char_df2['poem_count'] = author_df['author_poem_count'].sum()
char_df2

,author,title,char,char_id,author_id2,title_id2,author_poem_count,poem_char_count,poem_count
0,陶翰,經殺子谷,扶,0,16,561,7,60,1000
1,陶翰,經殺子谷,蘇,1,16,561,7,60,1000
2,陶翰,經殺子谷,秦,2,16,561,7,60,1000
3,陶翰,經殺子谷,帝,3,16,561,7,60,1000
4,陶翰,經殺子谷,子,4,16,561,7,60,1000
5,陶翰,經殺子谷,舉,5,16,561,7,60,1000
6,陶翰,經殺子谷,代,6,16,561,7,60,1000
7,陶翰,經殺子谷,稱,7,16,561,7,60,1000
8,陶翰,經殺子谷,其,8,16,561,7,60,1000
9,陶翰,經殺子谷,賢,9,16,561,7,60,1000


In [191]:
# char_df2['poem_id'] = char_df2['author_id2']

char_df2['book_id'] = 0
char_df2['book_id_pad'] = char_df2['book_id'].astype(str).str.zfill(2)
char_df2['author_id_pad'] = char_df2['author_id2'].astype(str).str.zfill(6)
char_df2['title_id_pad'] = char_df2['title_id2'].astype(str).str.zfill(7)
char_df2['char_id_pad'] = char_df2['char_id'].astype(str).str.zfill(4)
char_df2['poem_count_pad'] = char_df2['poem_count'].astype(str).str.zfill(7)
char_df2['author_poem_count_pad'] = char_df2['author_poem_count'].astype(str).str.zfill(7)
char_df2['poem_char_count_pad'] = char_df2['poem_char_count'].astype(str).str.zfill(4)


char_df2['char_genome'] = char_df2[[
                                    #total number of poems within the book
                                    'poem_count_pad',
                                    #total number of poems by the author within the book
                                   'author_poem_count_pad',
                                    #total number of characters within the poem
                                   'poem_char_count_pad',
                                    'book_id_pad', 
                                    'author_id_pad', 
                                    'title_id_pad', 
                                    'char_id_pad']].apply(lambda x: ''.join(x), axis=1)

char_df2 = char_df2.drop(['char_id', 'author_id2', 'title_id2',
                          'author_poem_count', 'poem_char_count',
                         'poem_count', 'book_id'], axis=1)

In [193]:
char_df2 = char_df2[['author', 'title', 'char', 'char_genome',
                     'poem_count_pad','author_poem_count_pad', 'poem_char_count_pad', # Count variables
                    'book_id_pad', 'author_id_pad', 'title_id_pad', 'char_id_pad']] # id's
char_df2

,author,title,char,char_genome,poem_count_pad,author_poem_count_pad,poem_char_count_pad,book_id_pad,author_id_pad,title_id_pad,char_id_pad
0,陶翰,經殺子谷,扶,0001000000000700600000001600005610000,0001000,0000007,0060,00,000016,0000561,0000
1,陶翰,經殺子谷,蘇,0001000000000700600000001600005610001,0001000,0000007,0060,00,000016,0000561,0001
2,陶翰,經殺子谷,秦,0001000000000700600000001600005610002,0001000,0000007,0060,00,000016,0000561,0002
3,陶翰,經殺子谷,帝,0001000000000700600000001600005610003,0001000,0000007,0060,00,000016,0000561,0003
4,陶翰,經殺子谷,子,0001000000000700600000001600005610004,0001000,0000007,0060,00,000016,0000561,0004
5,陶翰,經殺子谷,舉,0001000000000700600000001600005610005,0001000,0000007,0060,00,000016,0000561,0005
6,陶翰,經殺子谷,代,0001000000000700600000001600005610006,0001000,0000007,0060,00,000016,0000561,0006
7,陶翰,經殺子谷,稱,0001000000000700600000001600005610007,0001000,0000007,0060,00,000016,0000561,0007
8,陶翰,經殺子谷,其,0001000000000700600000001600005610008,0001000,0000007,0060,00,000016,0000561,0008
9,陶翰,經殺子谷,賢,0001000000000700600000001600005610009,0001000,0000007,0060,00,000016,0000561,0009


In [195]:
char_df2.to_csv(r'sample_data.csv', index=False)

## The length of a character genome now fits within the limit (64 hex)

In [181]:
hex(800006210086000100000000680087)

'0xa18f5a07cdb57d384b15ca097'

In [146]:
char_df[char_df['title']=='經殺子谷']

,author,title,char,char_id,author_code,title_code,author_id2,title_id2
0,陶翰,經殺子谷,扶,0,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
1,陶翰,經殺子谷,蘇,1,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
2,陶翰,經殺子谷,秦,2,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
3,陶翰,經殺子谷,帝,3,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
4,陶翰,經殺子谷,子,4,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
5,陶翰,經殺子谷,舉,5,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
6,陶翰,經殺子谷,代,6,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
7,陶翰,經殺子谷,稱,7,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
8,陶翰,經殺子谷,其,8,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561
9,陶翰,經殺子谷,賢,9,0xe999b6e7bfb0,0xe7b693e6aebae5ad90e8b0b7,16,561


In [23]:
# pageurl = "https://28utscprojects.wordpress.com/2011/01/21/001/"
pageurl = "https://raw.githubusercontent.com/chinese-poetry/chinese-poetry/master/json/poet.tang.7000.json"
print(pageurl)
browser = Browser('firefox')
browser.visit(pageurl)
soup = BeautifulSoup(browser.html, 'html.parser')    
soup

https://raw.githubusercontent.com/chinese-poetry/chinese-poetry/master/json/poet.tang.7000.json


<html><head><link href="resource://content-accessible/plaintext.css" rel="alternate stylesheet" title="Wrap Long Lines" type="text/css"/></head><body><pre>[
  {
    "strains": [
      "平平平仄仄，仄仄○○平。", 
      "仄仄○仄仄，仄平平仄平。", 
      "仄平仄仄仄，仄仄平仄平。", 
      "仄仄仄○仄，平平平仄○。", 
      "○平仄平仄，仄仄○平平。", 
      "仄仄仄平仄，平仄仄○平。"
    ], 
    "author": "陶翰", 
    "paragraphs": [
      "扶蘇秦帝子，舉代稱其賢。", 
      "百萬猶在握，可爭天下權。", 
      "束身就一劒，壯志皆棄捐。", 
      "塞下有遺跡，千齡人共傳。", 
      "疎蕪盡荒草，寂歷空寒煙。", 
      "到此盡垂淚，非我獨潸然。"
    ], 
    "title": "經殺子谷"
  }, 
  {
    "strains": [
      "仄平○仄平，平○○平仄。", 
      "平○仄仄仄，平仄仄平仄。", 
      "平平平仄○，仄仄平平仄。", 
      "平仄仄仄仄，仄平平○仄。", 
      "○平平仄仄，平仄仄平仄。", 
      "仄仄平仄平，平平仄平仄。", 
      "仄平仄仄仄，仄仄平○仄。"
    ], 
    "author": "陶翰", 
    "paragraphs": [
      "夜來三渚風，晨過臨淮島。", 
      "湖中海氣白，城上楚雲早。", 
      "鱗鱗魚浦帆，漭漭蘆洲草。", 
      "川路日浩蕩，惄焉心如擣。", 
      "且言任倚伏，何暇念枯槁。", 
      "范子名屢移，蘧公志常保。", 
      "古人去已久，此理今難道。"
    ], 
    "title": "早過臨淮"
  }, 
  {
    "strains": [
      "仄仄○仄平，平平○平仄。", 
 

In [10]:
soup.findAll('div', attrs={'class':'entry-content'})

[<div class="entry-content">
 <p></p>
 <p><span id="more-356"></span><br/><b>001 張九齡 – 感遇四首其一 <br/>001 ZHANG Jiuling – Thoughts I of IV</b></p>
 <p>中文原文﹕</p>
 <p>孤鴻海上來， <br/>池潢不敢顧。<br/>側見雙翠鳥， <br/>巢在三珠樹。</p>
 <p>矯矯珍木巔， <br/>得無金丸懼？<br/>美服患人指， <br/>高明逼神惡。</p>
 <p>今我游冥冥， <br/>弋者何所慕？</p>
 <p>英文翻譯 / English Translation﹕ </p>
 <p>A solitary swan flew in from the sea,<br/>Passing on ponds without thinking twice.<br/>Looking sideways it sees a couple of kingfishers of fine feathers,<br/>With their nest on a treasured tree of pearls enshrined</p>
 <p>Atop a tree of such eminence they perched daintily,<br/>Are they not afraid of powerful metal slingshot strikes?<br/>Well dressers draw attention,<br/>High positions attract the Deity's scrutiny for vice.</p>
 <p>As for I, much like that lone swan in the infinite sky,<br/>Wouldn't bird-hunters find me difficult to pinpoint and acquire? </p>
 <p>.</p>
 <div id="atatags-370373-5a88b890dd81c">
 <script type="text/javascript">
             __ATA.cmd.pu

In [13]:
soup.findAll('h1', attrs={'class':'entry-title'})[0].text

'001 張九齡 –\xa0感遇四首其一'

In [15]:
soup.find('h1', attrs={'class':'entry-title'}).text

'001 張九齡 –\xa0感遇四首其一'

In [18]:
soup.find('div', attrs={'class':'entry-content'}).findAll('p')

[<p></p>,
 <p><span id="more-356"></span><br/><b>001 張九齡 – 感遇四首其一 <br/>001 ZHANG Jiuling – Thoughts I of IV</b></p>,
 <p>中文原文﹕</p>,
 <p>孤鴻海上來， <br/>池潢不敢顧。<br/>側見雙翠鳥， <br/>巢在三珠樹。</p>,
 <p>矯矯珍木巔， <br/>得無金丸懼？<br/>美服患人指， <br/>高明逼神惡。</p>,
 <p>今我游冥冥， <br/>弋者何所慕？</p>,
 <p>英文翻譯 / English Translation﹕ </p>,
 <p>A solitary swan flew in from the sea,<br/>Passing on ponds without thinking twice.<br/>Looking sideways it sees a couple of kingfishers of fine feathers,<br/>With their nest on a treasured tree of pearls enshrined</p>,
 <p>Atop a tree of such eminence they perched daintily,<br/>Are they not afraid of powerful metal slingshot strikes?<br/>Well dressers draw attention,<br/>High positions attract the Deity's scrutiny for vice.</p>,
 <p>As for I, much like that lone swan in the infinite sky,<br/>Wouldn't bird-hunters find me difficult to pinpoint and acquire? </p>,
 <p>.</p>,
 <p class="jp-relatedposts-post jp-relatedposts-post0" data-post-format="false" data-post-id="353"><span class="jp-re

In [4]:
from bs4 import BeautifulSoup
# soup = BeautifulSoup(page.content, 'html.parser')

AttributeError: module 'html5lib.treebuilders' has no attribute '_base'